<h1 align="center">TPU - PRIMERA ENTREGA</h1>
<h3 align="center">Julian David Alfonso Moreno</h3>

---

## ¿Que es?
<p align="left">circuito integrado de aplicación específica y acelerador de IA (ASIC, AI accelerator application-specific integrated circuit) desarrollado por Google para el aprendizaje automático con redes neuronales artificiales<p>
<img width="300px" align="left" src="https://www.muycomputer.com/wp-content/uploads/2016/05/TPU-Google.jpg"/>

### Importamos las librerias

In [10]:
import numpy as np #para crear vectores y matrices grandes multidimensionales, funciones matemáticas de alto nivel
import os #propiedades del sistema operativo
import gzip #crea y extrae ficheros comprimidos

### Creamos la función  ````load_mnist(ruta, tipo='train')````

##### Lee los datos de la ruta especificada y los guarda en las variables etiquetas 

In [11]:
def load_mnist(ruta, tipo='train'):
    
    #cargamos las rutas(ubicaciones) de las categorias y las imagenes
    ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
    ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
    #abrimos el archivo de categorias(labels) en lectura binaria
    with gzip.open(ruta_categorias, 'rb') as rut_cat:
        #guardamos informacion en la variable en etiquetas
        etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)
    
    #abrimos el archivo de imagenes en lectura binaria
    with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
        #guardamos informacion en la variable en imagenes
        imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas),784)
        
    #devolvemos la informacion leida
    return imagenes, etiquetas

## Acceso a Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = 'gdrive/My Drive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_mnist(ruta, tipo='train')
X_test, Y_test = load_mnist(ruta, tipo='test')

## Reshape de los datos para garantizar que sean matrices (no vectores)

In [15]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

## Configuración TPU

In [ ]:
%tensorflow_version 2.x   # Para garantizar que la versión 2.x sea importada
import tensorflow as tf
print('Versión de TensorFlow: ' + tf.__version__)

tf.random.set_seed(200)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Creación del Modelo

In [17]:
with tpu_strategy.scope():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()

  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)        

## Entrenamiento con TPU



In [19]:
import timeit
x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

def entrenamiento_tpu():
  model.fit(x_train,y_train,validation_data=(x_test,y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 13s 16ms/step - loss: 0.6876 - accuracy: 0.7815 - val_loss: 0.4173 - val_accuracy: 0.8601
Epoch 2/2
468/468 [==============================] - 6s 14ms/step - loss: 0.3927 - accuracy: 0.8609 - val_loss: 0.3243 - val_accuracy: 0.8859


In [20]:
print('Tiempo de entrenamiento: ' + str(tpu_time) + ' segundos')

Tiempo de entrenamiento: 22.34910128699994 segundos
